In [55]:
import json
import pandas as pd
import requests
from tqdm import tqdm

In [69]:
API_KEY='c8025e44fb033c11205e040985474348'
# add link

In [72]:
df=pd.read_json('../DATA/TMDB_movie_ids_02_06_2022.json',lines=True)

In [73]:
def request_url(movie_id,
                API_KEY=API_KEY,
                lang='pt-BR'):
    req = 'https://api.themoviedb.org/3/movie/'+str(movie_id)+'?api_key='+API_KEY+'&language='+lang
    return(requests.get(req))

In [62]:
def process_request(json_file):
    file = json.loads(json_file.text)
    info = [file['adult'],
            [genre['name'] for genre in file['genres']],
            file['overview'],
            file['title'],
            file['vote_average'],
            file['vote_count'],
            file['runtime'],
            file['release_date']
           ]
    return(info)

In [ ]:
#info_list = [process_request(request_url(id)) for id in df.id[:3]]

info_list=[]
for movieid in tqdm(df.id):
    try:
        info_list.append(process_request(request_url(movieid,lang='en')))
        #info_list.append(request_url(movieid,lang='en'))
    except:
        info_list.append(None)
        print('Erro em '+str(movieid))

  4%|██▌                                                                   | 24545/677836 [1:28:04<38:54:26,  4.66it/s]

In [48]:
dfinfo = pd.DataFrame(info_list)
dfinfo.columns=['adult','genres','overview','title','vote_average',
                'vote_count','runtime','release_date']
dfinfo = pd.concat([dfinfo,pd.get_dummies(dfinfo['genres'].apply(pd.Series).stack()).sum(level=0)],axis=1)
dfinfo = dfinfo.drop(columns=['genres'])

In [ ]:
dfinfo.to_csv('TMDB_full.csv')

In [51]:
dfinfo2 = dfinfo[dfinfo.overview!='']
dfinfo2 = dfinfo2[~dfinfo2.overview.isnull()]
dfinfo2 = dfinfo2.reset_index().drop(columns=['index'])

In [ ]:
dfinfo2.to_csv('TMDB_subset.csv')